In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D

__Исходная задача:__ предсказание баллов ЕГЭ ученика в зависимости от кол-ва лет стажа его репетитора

In [2]:
X = np.array([[ 1,  1],
              [ 1,  1],
              [ 1,  2],
              [ 1,  5],
              [ 1,  3],
              [ 1,  0],
              [ 1,  5],
              [ 1, 10],
              [ 1,  1],
              [ 1,  2]])

In [3]:
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

In [4]:
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [5]:
W_analytical = np.linalg.inv((X.T @ X)) @ X.T @ y
W_analytical

array([45.0625,  3.8125])

### Домашнее задание <a class="anchor" id="hw"></a><center>

1. Подберите скорость обучения (eta) и количество итераций

In [6]:
n = X.shape[0]

# Найдя веса аналитическим путем во время вебинара, нам нужно стремиться к следующим значениям весов:
# W_analytical = array([45.0625,  3.8125])
# eta подбирал руками. Начиная с 120-ой итерации наша Mean Squared Error уже не меняется с точностью до двух знаков, 
# а веса достигают искомых аналитически найденных весов.
# ИЗМЕНЕНИЯ
eta = 0.055
n_iter = 220
# ИЗМЕНЕНИЯ

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    if i % 10 == 0:
        # ИЗМЕНЕНИЕ
        #eta /= 1.1 Это я закомментил вообще, оставив только принт ниже.
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err, 2)}')

Number of objects = 10        
Learning rate = 0.055        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [ 6.94  21.235], MSE = 3047.75
Iteration #10: W_new = [22.06405765 13.06063842], MSE = 848.44
Iteration #20: W_new = [31.21345509  8.7771262 ], MSE = 268.66
Iteration #30: W_new = [36.73513379  6.50786479], MSE = 109.6
Iteration #40: W_new = [40.06112387  5.29211526], MSE = 64.0
Iteration #50: W_new = [42.06149781  4.63338341], MSE = 50.33
Iteration #60: W_new = [43.26313928  4.2724736 ], MSE = 46.05
Iteration #70: W_new = [43.98427548  4.07261073], MSE = 44.67
Iteration #80: W_new = [44.41671151  3.96081125], MSE = 44.21
Iteration #90: W_new = [44.67586386  3.89768846], MSE = 44.05
Iteration #100: W_new = [44.83109194  3.86174724], MSE = 44.0
Iteration #110: W_new = [44.92403348  3.84112851], MSE = 43.98
Iteration #120: W_new = [44.97966334  3.82922172], MSE = 43.97
Iteration #130: W_new = [45.01295168  3.82230646], MSE = 43.97
Iteration #140: W_new = [45.03286686  3.818270

2*. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее

In [7]:
n = X.shape[0]

eta = 1e-2
n_iter = 100 # Чуть сократил кол-во итераций, чтобы было меньше принтов.

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
#     for k in range(W.shape[0]):
#         W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    # ИЗМЕНЕНИЯ
    # Не уверен, что в этом была задумка задания, но цикла больше нет. 
    W -= eta * (1/n * 2 * np.dot(X[:, 0], y_pred - y)), eta * (1/n * 2 * np.dot(X[:, 1], y_pred - y))
    # ИЗМЕНЕНИЯ
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

Iteration #0: W_new = [2.08 4.27], MSE = 3047.75
Iteration #10: W_new = [ 7.0011236 10.6169007], MSE = 738.65
Iteration #20: W_new = [10.3486292  10.10603105], MSE = 622.03
Iteration #30: W_new = [13.38789582  9.55618391], MSE = 525.24
Iteration #40: W_new = [16.16088505  9.05336203], MSE = 444.66
Iteration #50: W_new = [18.69110735  8.59454545], MSE = 377.58
Iteration #60: W_new = [20.99981865  8.17589626], MSE = 321.72
Iteration #70: W_new = [23.10641138  7.79389815], MSE = 275.22
Iteration #80: W_new = [25.02858024  7.44534246], MSE = 236.5
Iteration #90: W_new = [26.78247081  7.12730145], MSE = 204.27


3*. Вместо того, чтобы задавать количество итераций, задайте другое условие останова алгоритма - когда веса перестают изменяться меньше определенного порога $\epsilon$.

In [8]:
n = X.shape[0]

eta = 1e-2
#n_iter = 1000 Это комментируем, количество итераций нам больше не понадобится.


W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

# Заменяем for i in range на while, а так же создаем переменную limit, которая будет останавливать алгоритм,
# если между весами k-1 и k будет разница меньше чем limit.
limit = 0.5 # сделал лимит в 0.5, чтобы меньше текста было в аутпуте, но, естественно, значение можно менять.
current_limit = np.inf

while current_limit > limit:
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    # Взял формулу из предыдущего задания.
    new_W = W - (eta * (1/n * 2 * np.dot(X[:, 0], y_pred - y)), eta * (1/n * 2 * np.dot(X[:, 1], y_pred - y)))
    current_limit = np.linalg.norm(new_W - W, ord=2)
    W = new_W
    print(f'W_new = {W}, MSE = {round(err,2)}, current_limit={current_limit}')
print(f'Как видим, изменение между весами current_limit достигло лимита в {limit} и мы вышли из цикла')

Number of objects = 10        
Learning rate = 0.01        
Initial weights = [1.  0.5] 

W_new = [2.08 4.27], MSE = 3047.75, current_limit=3.921645062980585
W_new = [2.9122 6.6934], MSE = 1777.04, current_limit=2.562308412350083
W_new = [3.582352 8.242912], MSE = 1233.12, current_limit=1.6882212951055913
W_new = [4.14613024 9.2253808 ], MSE = 995.61, current_limit=1.1327360005186276
W_new = [4.63968479 9.83998351], MSE = 887.35, current_limit=0.7882465265551643
W_new = [ 5.08649208 10.21600803], MSE = 833.71, current_limit=0.5839787630698616
W_new = [ 5.50180176 10.43737578], MSE = 803.21, current_limit=0.4706227848573724
Как видим, изменение между весами current_limit достигло лимита в 0.5 и мы вышли из цикла
